## 事前準備

1. Google Cloud の新しいプロジェクトを作成します。
2. Cloud Shell を開いて、次のコマンドで API を有効化します。
```
gcloud services enable \
  aiplatform.googleapis.com \
  notebooks.googleapis.com \
  cloudresourcemanager.googleapis.com

```
3. 次のコマンドで Vertex AI Workbench のインスタンスを作成します。
```
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
gcloud workbench instances create agent-development \
  --project=$PROJECT_ID \
  --location=us-central1-a \
  --machine-type=e2-standard-2
```

4. クラウドコンソールのナビゲーションメニューから「Vertex AI」→「Workbench」を選択すると、作成したインスタンス agent-develpment があります。インスタンスの起動が完了するのを待って、「JUPYTERLAB を開く」をクリックしたら、「Python 3(ipykernel)」の新規ノートブックを作成します。

5. この後は、ノートブック上で以下のコマンドを実行していきます。

## ADK パッケージのインストール

In [ ]:
%pip install --upgrade --user google-adk

インストールしたパッケージを利用可能にするために、次のコマンドでカーネルを再起動します。

再起動を確認するポップアップが表示されるので [Ok] をクリックします。

In [2]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Search Agent App の作成

In [1]:
import json, os, pprint, time, uuid
import vertexai
from google.genai.types import Part, UserContent, ModelContent
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import google_search

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [2]:
instruction = '''
You are a friendly AI assistant that answers your queries.
Use google_search to give answers based on the latest and objective information.
'''

search_agent = LlmAgent(
    name='search_agent',
    model='gemini-2.0-flash-001',
    description='Agent to answer questions using Google Search.',
    instruction=instruction,
    tools=[google_search]
)

In [3]:
class LocalApp:
    def __init__(self, agent, app_name, user_id):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = self._runner.session_service.create_session(
            app_name=self._app_name,
            user_id=self._user_id,
            state={},
            session_id=uuid.uuid4().hex,
        )
        
    async def stream(self, query):
        content = UserContent(parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## 実行例

In [4]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

高田馬場でおすすめのカレー屋について調べてみました。


 高田馬場には、たくさんのカレー屋さんがありますね！おすすめのお店をいくつかご紹介します.

*   **ブラザー**：鯖キーマカレーが圧倒的な人気を誇るお店です。
*   **横浜ボンベイ**：サラッとしたルーが特徴で、カシミールカレーは長年愛される超有名カレーです。
*   **エチオピア**：トロトロに煮込まれた野菜が美味しいカレーが楽しめます。
*   **プネウマカレー**: コスパが良いと評判です.
*   **白カレーの店 1/f ゆらぎ**: 白カレーが人気です.

他にも、以下のお店がおすすめです。

*   ラモスカレー
*   アプサラ
*   まっちゃんカレー
*   マイカリー食堂 高田馬場店
*   カリーライス専門店 エチオピア 高田馬場店
*   ゴーゴーカレー 高田馬場駅前店

これらの情報を参考に、お好みのカレー屋さんを探してみてください.



SessionService が管理するセッション情報を確認します。

セッションを特定するのに必要な、`app_name`、`user_id`、`session_id` を確認します。

In [5]:
app_name = client._session.app_name
user_id = client._session.user_id
session_id = client._session.id

app_name, user_id, session_id

('Search Agent App', 'user00', '0a2753ea4f9348b783fc796e4bebd929')

確認した情報を使って、セッションに保存された情報を確認します。

2 つのイベント（ユーザーの質問とエージェントの回答）が記録されています。

エージェントの回答を生成する際に用いた Google Serch の情報なども確認できます。

In [6]:
session = client._runner.session_service.get_session(
    app_name = app_name,
    user_id = user_id,
    session_id = session_id,
)

session

Session(id='0a2753ea4f9348b783fc796e4bebd929', app_name='Search Agent App', user_id='user00', state={}, events=[Event(content=UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-9e95ac9d-d6d8-4dff-a683-bad9e11dc98a', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='zZlg6KP9', timestamp=1747325646.914193), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text

In [7]:
import datetime
def format_timestamp(ts):
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S.%f')

for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-15 16:14:06.914193 user
2025-05-15 16:14:06.915814 search_agent


続けて質問すると、セッション情報を踏まえた回答が得られます。

In [8]:
query = '''
特に家族連れにおすすめなのは？
'''
_ = await client.stream(query)

家族連れにおすすめの高田馬場のカレー屋さんについて調べてみます。


 家族連れにおすすめのカレー屋さんとしては、以下の点が考慮されるかと思います。

*   **子供向けメニュー**: 子供が食べやすい辛さのカレーや、カレー以外のメニューがあると嬉しいですね。
*   **お店の雰囲気**: 子連れでも気兼ねなく入れる、広々とした席や、子供が騒いでも大丈夫な雰囲気だと安心です。
*   **サービス**: 子供用の椅子や食器、離乳食の持ち込み可などのサービスがあると助かります。

これらの点を考慮すると、高田馬場でおすすめのカレー屋さんは以下の通りです。

*   **プネウマカレー**: こちらのお店は、小盛サイズがあるため、お子様にもおすすめです。店内はカウンター席ですが、ベビーカーを置けるスペースがあります。離乳食の持ち込みも可能で、土曜日にはお子様連れも来店するそうです。
*   **アプサラ**: こちらのお店はスリランカカレーのお店で、バナナリーフ包みがあります。バナナリーフ包みは、色々な味が混ざっていて子供の頃にご飯とカレーを混ぜて食べたような感覚で楽しめるとの口コミがありました。
*   **大戸屋ごはん処 高田馬場駅前店**: 大戸屋にはお子様野菜カレーがあり、小学生以下のお子様のみ注文できます。
*   **フードウェイ 高田馬場店**: こちらのお店は、家族連れの来店が多いです。

食べログなどのグルメサイトでは、子連れで利用しやすいカレー屋さんの情報が掲載されていますので、参考にしてみてください。



先ほどと同様にセッションに記録されたイベントを確認すると、新しい質問と回答のイベントが追加されています。

In [9]:
session = client._runner.session_service.get_session(
    app_name = app_name,
    user_id = user_id,
    session_id = session_id,
)

session

Session(id='0a2753ea4f9348b783fc796e4bebd929', app_name='Search Agent App', user_id='user00', state={}, events=[Event(content=UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-9e95ac9d-d6d8-4dff-a683-bad9e11dc98a', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='zZlg6KP9', timestamp=1747325646.914193), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text

In [10]:
for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-15 16:14:06.914193 user
2025-05-15 16:14:06.915814 search_agent
2025-05-15 16:14:10.553314 user
2025-05-15 16:14:10.554543 search_agent


## システムインストラクションの確認

ちょっとした裏技を使って、LlmAgent オブジェクトが Gemini API に送っている生の情報を覗いてみます。

In [11]:
import pprint
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator
from google.adk.events.event import Event
from google.adk.models.llm_request import LlmRequest

async def _run_one_step_async2(
      self,
      invocation_context: InvocationContext,
  ) -> AsyncGenerator[Event, None]:
    """One step means one LLM call."""
    llm_request = LlmRequest()

    # Preprocess before calling the LLM.
    async for event in self._preprocess_async(invocation_context, llm_request):
        yield event
    if invocation_context.end_invocation:
        return

    # Calls the LLM.
    model_response_event = Event(
        id=Event.new_id(),
        invocation_id=invocation_context.invocation_id,
        author=invocation_context.agent.name,
        branch=invocation_context.branch,
    )

    ## DEBUG output
    if DEBUG:
        print('## Prompt contents ##')
        pprint.pp(llm_request.contents)
        print('----')
        print('## System instruction ##')
        print(llm_request.config.system_instruction)
        print('----')
        print('## Tools config ##')
        pprint.pp(llm_request.config.tools)
        print('----')
    ####

    async for llm_response in self._call_llm_async(
        invocation_context, llm_request, model_response_event
    ):
        # Postprocess after calling the LLM.
        async for event in self._postprocess_async(
            invocation_context, llm_request, llm_response, model_response_event
        ):
            yield event

from google.adk.flows.llm_flows.base_llm_flow import BaseLlmFlow
BaseLlmFlow._run_one_step_async = _run_one_step_async2

In [12]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

DEBUG = True
query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user')]
----
## System instruction ##

You are a friendly AI assistant that answers your queries.
Use google_search to give answers based on the latest and objective information.


You are an agent. Your internal name is "search_agent".

 The description about you is "Agent to answer questions using Google Search."
----
## Tools config ##
[Tool(retrieval=None, google_search=GoogleSearch(), google_search_retrieval=None, enterprise_web_search=None, google_maps=None, code_execution=None, function_declarations=None)]
----
 高田馬場でおすすめのカレー屋について調べてみました。


 高田馬場にはたくさんのカレー屋さんがあり、それぞれに特色があります。おすすめのカレー店をいくつかご紹介します.

*   **カレーライス専門店ブラザー:** 鯖キーマカレーが人気のお店です。
*   **横浜ボンベイ 高田馬場店:** ボンベイカレーやカシミールカレーが人気で、特にカシミールカレーは激辛ながらも野菜の甘みとスパイスの爽快感が味わえることで有名です。
*   **プネウマカレー

In [13]:
query = '''
特に家族連れにおすすめなのは？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text=' 高田馬場でおすすめのカレー屋について調べてみました。\n\n'), Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text=' 高田馬場にはたくさんのカレー屋さんがあり、それぞれに特色があります。おすすめのカレー店をいくつかご紹介します.\n\n*   **カレーライス専門店ブラザー:** 鯖キーマカレーが人気のお店です。\n*   **横浜ボンベイ 高田馬場店:** ボンベイカレーやカシミールカレーが人気で、特にカシミールカレーは激辛ながらも野菜の甘みとスパイスの爽快感が味わえることで有名です。\n*   **プネウマカレー:** チキンカレーが人気で、食べログのカレーTOKYO百名店にも選出されています。\n*   **エチオピア 高田馬場店:** 野菜カレーが人気で、トロトロに煮込まれた野菜が美味しいと評判です。\n*   **白カレーの店 1/f ゆらぎ:** その名の通り、白いカレーが楽しめるお店

`## Prompt contents ##` の部分を見ると、セッションに記録されたイベントをすべてプロンプトに含めるのではなく、コンテキスト（これまでの会話の流れ）を把握するのに必要な情報が選択されていることがわかります。

## AgentEngine へのデプロイ

In [22]:
from vertexai import agent_engines

remote_agent = agent_engines.create(
    agent_engine=search_agent,
    requirements=[
        'google-cloud-aiplatform[adk,agent_engines]',
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.88.0'}
The following requirements are missing: {'cloudpickle'}
The following requirements are appended: {'cloudpickle==3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'cloudpickle==3.1.1']
Using bucket etsuji-15pro-poc
Wrote to gs://etsuji-15pro-poc/agent_engine/agent_engine.pkl
Writing to gs://etsuji-15pro-poc/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://etsuji-15pro-poc/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/879055303739/locations/us-central1/reasoningEngines/193311736349065216/operations/7049101365024391168
View progress and logs at https://console.cloud.google.com/logs/query?project=etsuji-15pro-poc
AgentEngine created. Resource name: projects/879055303739/locations/us-central1/reasoningEngines/1933

In [23]:
class RemoteApp:
    def __init__(self, remote_agent, user_id):
        self._remote_agent = remote_agent
        self._user_id = user_id
        self._session = remote_agent.create_session(user_id=self._user_id)
    
    def _stream(self, query):
        events = self._remote_agent.stream_query(
            user_id=self._user_id,
            session_id=self._session['id'],
            message=query,
        )
        result = []
        for event in events:
            if DEBUG:
                print(f'----\n{event}\n----')
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

    def stream(self, query):
        # Retry 4 times in case of resource exhaustion 
        for c in range(4):
            if c > 0:
                time.sleep(2**(c-1))
            result = self._stream(query)
            if result:
                return result
            if DEBUG:
                print('----\nRetrying...\n----')
        return None # Permanent error

In [24]:
remote_client = RemoteApp(remote_agent, 'user00')
DEBUG=False

query = '''
今年のゴールデンウィークは、カレンダー通りで何連休でしょうか？
'''
_ = remote_client.stream(query)

今年のゴールデンウィークがカレンダー通りだとすると、何連休になるか調べてみます。


 2025年のゴールデンウィークは、カレンダー通りだと以下のようになります:

*   4月26日(土)～29日(火・祝)で4連休。
*   5月3日(土・祝)～6日(火・振休)で4連休。

カレンダー通りにお休みの場合、飛び石連休となり、4連休が2回となります。4月30日(水)、5月1日(木)、2日(金)の3日間を休むことができれば、4月29日(火・祝)から5月7日(火・振休)まで、8連休にすることも可能です。



In [25]:
query = '''
来年はどうなりますか？
'''
_ = remote_client.stream(query)

来年、2026年のゴールデンウィークがカレンダー通りだと何連休になるか調べてみます。


 来年2026年のゴールデンウィークは、カレンダー通りだと5連休になります.

*   5月2日(土)～5月6日(水・振替休日)まで.

4月30日(木)と5月1日(金)に有給休暇を取得すると、4月29日(水)の昭和の日から5月6日(水)まで、8連休にすることも可能です.



セッション情報を参照する際は、`VertexAiSessionService` を使用します。

In [26]:
app_name = remote_client._session['app_name']
user_id = remote_client._session['user_id']
session_id = remote_client._session['id']

app_name, user_id, session_id

('193311736349065216', 'user00', '1378572626707873792')

In [27]:
from google.adk.sessions import VertexAiSessionService

session_service = VertexAiSessionService(
    project = PROJECT_ID,
    location = LOCATION,
)

session = session_service.get_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id,
)

session

Session(id='1378572626707873792', app_name='193311736349065216', user_id='user00', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n今年のゴールデンウィークは、カレンダー通りで何連休でしょうか？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-96f830d4-83d4-4d68-b354-681072162b53', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='6962737097484533760', timestamp=1747326181.287548), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response

In [28]:
for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-15 16:23:01.287548 user
2025-05-15 16:23:01.355285 search_agent
2025-05-15 16:23:04.341686 user
2025-05-15 16:23:04.407377 search_agent


## 後片付け

デプロイ済みのエージェントをまとめて削除します。

In [29]:
for agent in agent_engines.list():
    print(agent.gca_resource.name)
    agent.delete(force=True)

projects/879055303739/locations/us-central1/reasoningEngines/193311736349065216
Delete Agent Engine backing LRO: projects/879055303739/locations/us-central1/operations/6509795309646774272
Agent Engine deleted. Resource name: projects/879055303739/locations/us-central1/reasoningEngines/193311736349065216
